In [37]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sqlalchemy import create_engine, inspect

In [38]:
# movies_df = pd.read_csv('data/movies_model.csv')
# movies_df

In [39]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("data/movies_model.sqlite")
# Read sqlite query results into a pandas DataFrame
movies_df = pd.read_sql_query("SELECT * from movies", con)

# Verify that result of SQL query is stored in the dataframe
con.close()
movies_df

,genre,score,writer,star,budget,gross,company,runtime
0,Drama,8.4,Stephen King,Jack Nicholson,19000000,46998772,Warner Bros.,146
1,Adventure,5.8,Henry De Vere Stacpoole,Brooke Shields,4500000,58853106,Columbia Pictures,104
2,Action,8.7,Leigh Brackett,Mark Hamill,18000000,538375067,Lucasfilm,124
3,Comedy,7.7,Jim Abrahams,Robert Hays,3500000,83453539,Paramount Pictures,88
4,Comedy,7.3,Brian Doyle-Murray,Chevy Chase,6000000,39846344,Orion Pictures,98
...,...,...,...,...,...,...,...,...
5416,Action,6.6,Peter Craig,Will Smith,90000000,426505244,Columbia Pictures,124
5417,Action,6.5,Pat Casey,Ben Schwartz,85000000,319715683,Paramount Pictures,99
5418,Adventure,5.6,Stephen Gaghan,Robert Downey Jr.,175000000,245487753,Universal Pictures,101
5419,Adventure,6.8,Michael Green,Harrison Ford,135000000,111105497,20th Century Studios,100


In [40]:
movies_y = movies_df['gross']
movies_y

0        46998772
1        58853106
2       538375067
3        83453539
4        39846344
          ...    
5416    426505244
5417    319715683
5418    245487753
5419    111105497
5420    461421559
Name: gross, Length: 5421, dtype: int64

In [41]:
writer_counts = movies_df['writer'].value_counts()
star_counts = movies_df['star'].value_counts()
company_counts = movies_df['company'].value_counts()

In [42]:
movies_other= movies_df.apply(lambda x: x.mask(x.map(x.value_counts())<25, 'other') if x.name=='star' else x)
movies_other = movies_other.apply(lambda x: x.mask(x.map(x.value_counts())< 50, 'other') if x.name=='company' else x)
movies_other= movies_other.apply(lambda x: x.mask(x.map(x.value_counts())<100, 'other') if x.name=='genre' else x)
movies_other= movies_other.apply(lambda x: x.mask(x.map(x.value_counts())<12, 'other') if x.name=='writer' else x)
movies_other

,genre,score,writer,star,budget,gross,company,runtime
0,Drama,8.4,Stephen King,other,19000000,46998772,Warner Bros.,146
1,Adventure,5.8,other,other,4500000,58853106,Columbia Pictures,104
2,Action,8.7,other,other,18000000,538375067,other,124
3,Comedy,7.7,other,other,3500000,83453539,Paramount Pictures,88
4,Comedy,7.3,other,other,6000000,39846344,other,98
...,...,...,...,...,...,...,...,...
5416,Action,6.6,other,other,90000000,426505244,Columbia Pictures,124
5417,Action,6.5,other,other,85000000,319715683,Paramount Pictures,99
5418,Adventure,5.6,other,other,175000000,245487753,Universal Pictures,101
5419,Adventure,6.8,other,Harrison Ford,135000000,111105497,other,100


In [43]:
movies_x = movies_other.drop(columns=['gross'])
movies_x

,genre,score,writer,star,budget,company,runtime
0,Drama,8.4,Stephen King,other,19000000,Warner Bros.,146
1,Adventure,5.8,other,other,4500000,Columbia Pictures,104
2,Action,8.7,other,other,18000000,other,124
3,Comedy,7.7,other,other,3500000,Paramount Pictures,88
4,Comedy,7.3,other,other,6000000,other,98
...,...,...,...,...,...,...,...
5416,Action,6.6,other,other,90000000,Columbia Pictures,124
5417,Action,6.5,other,other,85000000,Paramount Pictures,99
5418,Adventure,5.6,other,other,175000000,Universal Pictures,101
5419,Adventure,6.8,other,Harrison Ford,135000000,other,100


In [44]:
movies_x_dummies = pd.get_dummies(movies_x)
dummy_columns = movies_x_dummies.columns.values
dummy_columns

array(['score', 'budget', 'runtime', 'genre_Action', 'genre_Adventure',
       'genre_Animation', 'genre_Biography', 'genre_Comedy',
       'genre_Crime', 'genre_Drama', 'genre_Horror', 'genre_other',
       'writer_Joel Coen', 'writer_John Hughes', 'writer_Luc Besson',
       'writer_Stephen King', 'writer_Wes Craven', 'writer_Woody Allen',
       'writer_other', 'star_Adam Sandler', 'star_Arnold Schwarzenegger',
       'star_Bruce Willis', 'star_Denzel Washington', 'star_Eddie Murphy',
       'star_Harrison Ford', 'star_John Travolta', 'star_Johnny Depp',
       'star_Keanu Reeves', 'star_Kevin Costner', 'star_Matt Damon',
       'star_Matthew McConaughey', 'star_Mel Gibson', 'star_Nicolas Cage',
       'star_Robert De Niro', 'star_Sylvester Stallone',
       'star_Tom Cruise', 'star_Tom Hanks', 'star_other',
       'company_Columbia Pictures', 'company_Dimension Films',
       'company_Dreamworks Pictures', 'company_Fox 2000 Pictures',
       'company_Lionsgate', 'company_Metro-Gold

In [45]:
X_train, X_test, y_train, y_test = train_test_split(movies_x_dummies, movies_y, random_state=1)

In [46]:
model = RandomForestRegressor(criterion="mse")

In [47]:
model.fit(X_train,y_train)

RandomForestRegressor()

In [48]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

predicted = model.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
mae = mean_absolute_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"mean squared error (MSE): {mse}")
print(f"mean absolute error (MAE): {mae}")
print(f"R-squared (R2): {r2}")


mean squared error (MSE): 1.4286680461800464e+16
mean absolute error (MAE): 62376971.278483376
R-squared (R2): 0.5844238931451284


In [49]:
import pickle
filename = 'movies_model.sav'
pickle.dump(model, open(filename, 'wb'))